<a href="https://colab.research.google.com/github/SahSanjeev/Chat/blob/main/Streamlit_Motor_NYC_April.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

rhyme@ip-10-199-52-70:~/Desktop/Project$ atom .
rhyme@ip-10-199-52-70:~/Desktop/Project$ streamlit run app.py

  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://10.199.52.70:8501


In [1]:
! pip install streamlit
import streamlit as st
st.title("Hello World !Woooo")
st.markdown("My First Streamlit Dashboard ")
import pandas as pd
import numpy as np
import pydeck as pdk
import plotly.express as px
DATA_URL = (
"/content/Motor_Vehicle_Collisions_-_Crashes_20240409.csv"
)
st.title("Motor Vechicle Collisions in New York City")
st.markdown("This application is a streamlit dashboard to analyze Motor Vechicle Collision in NY City")

@st.cache(persist=True)
def load_data(nrows):
    data = pd.read_csv(DATA_URL,nrows=nrows, parse_dates=[['CRASH_DATE', 'CRASH_TIME']])
    data.dropna(subset=['LATITUDE','LONGITUDE'] , inplace = True)
    lowercase =lambda x:str(x).lower()
    data.rename(lowercase , axis ='columns', inplace = True)
    data.rename(columns={'crash_date_crash_time': 'date/time'},inplace =True)
    return data
data = load_data(100000)
st.header("Where are the most people injured in NYC ?")
injured_people = st.slider("Number of persons injured in vehicle col" , 0, 19)
st.map(data.query("injured_persons >= @injured_people ")[["latitude","longitude"]].dropna(how="any"))


st.header("How many collisions occure during a given time of day ?")
hour = st.slider("Hour to look at ", 0,23)
data =data[data['date/time'].dt.hour == hour]

st.markdown("Vehicle collisions between %i:00 and %i:00" % (hour, (hour+1) % 24))

midpoint =(np.average(data['latitude']), np.average(data['longitude']))

st.write(pdk.Deck(
 map_style ="mapbox://styles/mapbox/light-v9" ,
 initial_view_state ={
  "latitude":midpoint[0],
  "longitude":midpoint[1],
  "zoom":11,
  "pitch":50,
 },
 layers =[
 pdk.Layer("HexagonLayer",
 data = data[['date/time', 'latitude','longitude']],
 get_position=['longitude','latitude'],
 radius =100,
 extruded = True ,
 pickable = True ,
 elevation_scale = 4,
 elevation_range = [0,1000],
 ),
 ],
))

st.subheader("Braekdown by minute between %i:00 and %i:00" % (hour ,(hour+1) %  24))
filtered = data[
(data['date/time'].dt.hour >= hour ) & (data['date/time'].dt.hour < (hour+1))
]

hist = np.histogram(filtered['date/time'].dt.minute,bins =60, range=(0,60))[0]
chart_data = pd.DataFrame({'minute': range(60),'crashes':hist})
fig =px.bar(chart_data,x ='minute',y='crashes',hover_data=['minute','crashes'],height =400)
st.write(fig)

st.header("Top 5 dangerous street by affected type ")
select = st.selectbox('Affected type of people',['Pedestrians','Cyclists','Motorist'])
if select =='Pedestrians':
    st.write(original_data.query('injured_pedestrians >= 1')[['on_street_name','injured_pedestrians']].sort_values(by=['injured_pedestrians'],ascending =False).dropna(how='any')[:5])


elif select == 'Cyclists':
    st.write(original_data.query('injured_cyclists >= 1')[['on_street_name','injured_cyclists']].sort_values(by=['injured_cyclists'],ascending =False).dropna(how='any')[:5])

else:
    st.write(original_data.query('injured_motorists >= 1')[['on_street_name','injured_motorists']].sort_values(by=['injured_motorists'],ascending =False).dropna(how='any')[:5])





if st.checkbox("Show raw Data", False):
    st.subheader('Raw Data')
    st.write(data)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.3 MB/s eta 0:00:00


2024-04-18 05:45:14.170 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]


InternalHashError: module '__main__' has no attribute '__file__'

While caching the body of `load_data()`, Streamlit encountered an
object of type `builtins.function`, which it does not know how to hash.

**In this specific case, it's very likely you found a Streamlit bug so please
[file a bug report here.]
(https://github.com/streamlit/streamlit/issues/new/choose)**

In the meantime, you can try bypassing this error by registering a custom
hash function via the `hash_funcs` keyword in @st.cache(). For example:

```
@st.cache(hash_funcs={builtins.function: my_hash_func})
def my_func(...):
    ...
```

If you don't know where the object of type `builtins.function` is coming
from, try looking at the hash chain below for an object that you do recognize,
then pass that to `hash_funcs` instead:

```
Object of type builtins.function: <function load_data at 0x7e7217e8e320>
```

Please see the `hash_funcs` [documentation](https://docs.streamlit.io/library/advanced-features/caching#the-hash_fun